In [1]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()

... Checking Tensorflow Version ...
Tensorflow built with CUDA? True
TF: 2.10.0 | CUDA: 64_112 | cuDNN: 64_8
# GPU available: 1 (NVIDIA GeForce RTX 3080)


In [2]:
proxy.load_data()
proxy.process_data(subsample=600)

model0 = proxy.make_model()
model0.summary()

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
normalized - X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
Subsampling data for 600 samples ...
Train - X: (450, 64, 64, 4) | y: (450, 60, 64, 64, 2)
Test  - X: (150, 64, 64, 4) | y: (150, 60, 64, 64, 2)
# Parameters: 340,162
Model: "CNN-RNN-Proxy"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 4)]       0         
                                                                 
 Encoder (Functional)        (None, 8, 8, 128)         34308     
                                                                 
 Dynamic (Functional)        (None, 60, 8, 8, 128)     50880     
                                                                 
 Decoder (Functional)        (None, 60, 64, 64, 2)     254974    
                                                                 
Total params: 340,162
Trainable params: 3

In [3]:
model, fit = proxy.train(model0)

ResourceExhaustedError: Graph execution error:

Detected at node 'CNN-RNN-Proxy/Decoder/instance_normalization_5/moments/SquaredDifference' defined at (most recent call last):
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Misael Morales

In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
k = 258

plt.figure(figsize=(20,2))
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.imshow(proxy.X_train[k,:,:,i], 'jet')
    plt.xticks([]); plt.yticks([])
plt.show()

fig, axs = plt.subplots(2, 12, figsize=(20,4))
for j in range(12):
    axs[0,j].imshow(proxy.y_train[k,j*5,:,:,0], 'jet')
    axs[1,j].imshow(y_train_pred[k,j*5,:,:,0], 'jet')
    axs[0,j].set(title='t={}'.format(j*5))
    for i in range(2):
        axs[i,j].set(xticks=[], yticks=[])
axs[0,0].set(ylabel='True'); axs[1,0].set(ylabel='Pred')
plt.show()

fig, axs = plt.subplots(2, 12, figsize=(20,4))
for j in range(12):
    axs[0,j].imshow(proxy.y_train[k,j*5,:,:,1], 'jet')
    axs[1,j].imshow(y_train_pred[k,j*5,:,:,1], 'jet')
    axs[0,j].set(title='t={}'.format(j*5))
    for i in range(2):
        axs[i,j].set(xticks=[], yticks=[])
axs[0,0].set(ylabel='True'); axs[1,0].set(ylabel='Pred')
plt.show()

***
# END